In [13]:
from tools import *
from tqdm import tqdm
from typing import List, Dict
import json
import requests
from tqdm import tqdm
from typing import List, Dict,Tuple
import json
import requests
import tools


In [4]:
stats = tools.clear_all_graph_data()


🗑️  Starting to clear all graph data...
   Deleting all relationships...
   Deleting all nodes...
   Cleaning up GDS graphs...
   - No 'entity' graph to drop
   Dropping all indexes...
   Dropping all constraints...
✅ Graph cleanup completed!
   - Deleted 0 relationships
   - Deleted 0 nodes
   - Remaining nodes: 0


In [5]:
url = "https://www.gutenberg.org/cache/epub/1727/pg1727.txt"
response = requests.get(url)

In [6]:
def chunk_into_books(text: str) -> List[str]:
    return (
        text.split("PREFACE TO FIRST EDITION")[2]
        .split("FOOTNOTES")[0]
        .strip()
        .split("\nBOOK")[1:]
    )

books = chunk_into_books(response.text)

In [7]:
token_count = [num_tokens_from_string(el) for el in books]
print(
    f"""There are {len(token_count)} books with token sizes:
- avg {sum(token_count) / len(token_count)}
- min {min(token_count)}
- max {max(token_count)}
"""
)

There are 24 books with token sizes:
- avg 6515.208333333333
- min 4459
- max 10760



In [8]:
chunked_books = [chunk_text(book, 1000, 40) for book in books]

In [14]:
#이미 정해져 있는 entity
ENTITY_TYPES = "PERSON,ORGANIZATION,LOCATION,GOD,EVENT,CREATURE,WEAPON_OR_TOOL"

def extract_entities(text: str) -> Tuple[List[Dict], List[Dict]]:
    """
    Extract entities and relationships from text using Bedrock
    
    Args:
        text: Input text to analyze
        
    Returns:
        Tuple of (entities, relationships) as lists of dictionaries
    """
    # Construct prompt
    messages = [
        {"role": "user", "content": tools.create_extraction_prompt(ENTITY_TYPES, text)}
    ]
    
    # Make the LLM call using Bedrock (model parameter is ignored)
    output = tools.chat(messages, model="gpt-4o")  # Will use Bedrock instead
    
    # Parse output and return both entities and relationships
    return tools.parse_extraction_output(output)

In [16]:
number_of_books = 1
for book_i, book in enumerate(
    tqdm(chunked_books[:number_of_books], desc="Processing Books")
):
    for chunk_i, chunk in enumerate(tqdm(book, desc=f"Book {book_i}", leave=False)):
        nodes, relationships = extract_entities(chunk)
        neo4j_driver.execute_query(
            tools.import_nodes_query,
            data=nodes,
            book_id=book_i,
            text=chunk,
            chunk_id=chunk_i,
        )
        neo4j_driver.execute_query(
            tools.import_relationships_query, data=relationships
        )

Book 0:   0%|          | 0/22 [00:00<?, ?it/s]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";MUSE;GOD;Divine being invoked to tell the story of the hero's journey, referred to as daughter of Jove)
|
("entity";TROY;LOCATION;Famous town that was sacked by the ingenious hero)
|
("entity";HYPERION;GOD;Sun-god whose cattle were eaten by the hero's men, leading to their destruction and preventing them from reaching home)
|
("entity";JOVE;GOD;Father deity whose daughter is the Muse)
|
("entity";ULYSSES;PERSON;Ingenious hero who travelled far and wide after sacking Troy, longing to return to his wife and country but detained by Calypso)
|
("entity";CALYPSO;GOD;Goddess who detained Ulysses in a large cave and wanted to marry him)
|
("entity";MINERVA;GOD;Goddess mentioned in the chapter title who visits Ithaca)
|
("entity";ITHACA;LOCATION;Location visited by Minerva as mentioned in the chapter title)
|
("entity";TELEMACHUS;PERSON;Person who challenges the suitors as mentioned in the chapter title)
|
("ent


Book 0:   5%|▍         | 1/22 [00:09<03:20,  9.56s/it]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";ITHACA;LOCATION;Island home where the hero is destined to return after being settled by the gods)
|
("entity";NEPTUNE;GOD;God who persecuted the hero without ceasing and would not let him get home, went to the Ethiopians to accept sacrifices)
|
("entity";ETHIOPIANS;PERSON;People at the world's end who lie in two halves, one looking West and the other East, who offered sacrifices to Neptune)
|
("entity";JOVE;GOD;Olympian god, sire of gods and men, who spoke first when the gods met in his house)
|
("entity";AEGISTHUS;PERSON;Man who made love to Agamemnon's wife unrighteously and killed Agamemnon, later killed by Orestes)
|
("entity";AGAMEMNON;PERSON;Man who was killed by Aegisthus, father of Orestes)
|
("entity";ORESTES;PERSON;Son of Agamemnon who killed Aegisthus)
|
("entity";OLYMPUS;LOCATION;Location of Jove's house where the gods met)
|
("relationship";NEPTUNE;ITHACA;Neptune prevented the hero from reac


Book 0:   9%|▉         | 2/22 [00:19<03:10,  9.52s/it]

...


Book 0:  91%|█████████ | 20/22 [04:13<00:24, 12.44s/it]

 stranger possibly brought news about Ulysses' return;6)⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";TELEMACHUS;PERSON;Young man who believes his father is dead, puts no faith in rumors, knows the stranger was actually the goddess, retires to his high tower room)
|
("entity";ULYSSES;PERSON;Dead father of Telemachus, old friend of Mentes according to Telemachus' explanation)
|
("entity";PENELOPE;PERSON;Telemachus' mother who sometimes sends for a soothsayer and questions him about prophecies)
|
("entity";SOOTHSAYER;PERSON;Prophet whom Penelope sometimes sends for and questions, but whose prophecies Telemachus ignores)
|
("entity";MENTES;PERSON;Identity claimed for the stranger, son of Anchialus and chief of the Taphians, old friend of Ulysses)
|
("entity";ANCHIALUS;PERSON;Father of Mentes according to Telemachus' explanation)
|
("entity";TAPHIANS;PERSON;People led by Mentes as their chief)
|
("entity";MINERVA;GOD;Goddess who was actually the strange


Book 0:  95%|█████████▌| 21/22 [04:29<00:13, 13.73s/it]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
("entity";LAERTES;PERSON;Man who had a wedded wife and feared her resentment, so did not take Euryclea to his bed despite showing her respect)
|
("entity";LAERTES' WIFE;PERSON;Wedded wife of Laertes whose resentment he feared, preventing him from taking Euryclea to his bed)
|
("entity";EURYCLEA;PERSON;Good old woman who lighted Telemachus to his room, loved him better than other women in the house, had nursed him as a baby, helped him prepare for bed)
|
("entity";TELEMACHUS;PERSON;Young man who was nursed by Euryclea as a baby, goes to his bedroom, takes off his shirt, lies covered with woollen fleece thinking of his voyage)
|
("entity";MINERVA;GOD;Goddess who gave counsel to Telemachus about his intended voyage)
|
("entity";BEDROOM;LOCATION;Telemachus' private room where he sleeps and prepares for bed)
|
("entity";BED;WEAPON_OR_TOOL;Furniture where Telemachus sits and later lies down to sleep)
|
("entity";SHIRT;W


Processing Books: 100%|██████████| 1/1 [04:45<00:00, 285.80s/it]


In [17]:
nodes

[{'record_type': 'entity',
  'entity_name': 'LAERTES',
  'entity_type': 'PERSON',
  'entity_description': 'Man who had a wedded wife and feared her resentment, so did not take Euryclea to his bed despite showing her respect'},
 {'record_type': 'entity',
  'entity_name': "LAERTES' WIFE",
  'entity_type': 'PERSON',
  'entity_description': 'Wedded wife of Laertes whose resentment he feared, preventing him from taking Euryclea to his bed'},
 {'record_type': 'entity',
  'entity_name': 'EURYCLEA',
  'entity_type': 'PERSON',
  'entity_description': 'Good old woman who lighted Telemachus to his room, loved him better than other women in the house, had nursed him as a baby, helped him prepare for bed'},
 {'record_type': 'entity',
  'entity_name': 'TELEMACHUS',
  'entity_type': 'PERSON',
  'entity_description': 'Young man who was nursed by Euryclea as a baby, goes to his bedroom, takes off his shirt, lies covered with woollen fleece thinking of his voyage'},
 {'record_type': 'entity',
  'entity_

In [18]:
relationships

[{'record_type': 'relationship',
  'source_entity': 'LAERTES',
  'target_entity': "LAERTES' WIFE",
  'relationship_description': 'Laertes is married to his wedded wife and fears her resentment',
  'relationship_strength': 9},
 {'record_type': 'relationship',
  'source_entity': 'LAERTES',
  'target_entity': 'EURYCLEA',
  'relationship_description': "Laertes showed respect to Euryclea but did not take her to his bed due to fear of his wife's resentment",
  'relationship_strength': 8},
 {'record_type': 'relationship',
  'source_entity': 'EURYCLEA',
  'target_entity': 'TELEMACHUS',
  'relationship_description': 'Euryclea nursed Telemachus as a baby and loves him better than other women in the house',
  'relationship_strength': 9},
 {'record_type': 'relationship',
  'source_entity': 'TELEMACHUS',
  'target_entity': 'BEDROOM',
  'relationship_description': 'Telemachus goes to his bedroom to prepare for sleep',
  'relationship_strength': 8},
 {'record_type': 'relationship',
  'source_entity':

In [ ]:
for k in 

In [11]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (:`__Entity__`)
    RETURN 'entity' AS type, count(*) AS count
    UNION
    MATCH ()-[:RELATIONSHIP]->()
    RETURN 'relationship' AS type, count(*) AS count
    """
)
print([el.data() for el in data])

[{'type': 'entity', 'count': 114}, {'type': 'relationship', 'count': 196}]


In [12]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (n:PERSON)
WHERE n.name = "ORESTES"
RETURN n.description AS description"""
)
print([el.data()['description'] for el in data])

[["Son of Agamemnon who killed Aegisthus in revenge for his father's murder", 'Son who would grow up to take revenge on Aegisthus for killing his father Agamemnon', "Hero praised for killing his father's murderer Aegisthus, held up as an example for Telemachus"]]


In [13]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (n:__Entity__)-[:RELATIONSHIP]-(m:__Entity__)
WITH n,m, count(*) AS countOfRels
ORDER BY countOfRels DESC LIMIT 1
MATCH (n)-[r:RELATIONSHIP]-(m)
RETURN n.name AS source, m.name AS target, countOfRels, collect(r.description) AS descriptions
"""
)
print([el.data() for el in data])

[{'source': 'TELEMACHUS', 'target': 'ULYSSES', 'countOfRels': 13, 'descriptions': ['Telemachus is the son of Ulysses and mourns his father as dead and gone', 'Telemachus is the son of Ulysses and will rule those whom his father won for him', "Telemachus is the son of Ulysses and refers to his father's house", 'Telemachus is seeking news of his missing father Ulysses', 'Telemachus should go in quest of his father Ulysses who has long been missing', 'Telemachus is the son of Ulysses', 'Telemachus is the son of Ulysses and thinks about his brave father', 'Telemachus is the son of Ulysses, referred to as his unhappy father', 'Telemachus refers to his absent father whose bones may be rotting, indicating Ulysses is his father', 'Telemachus is the son of Ulysses and speaks about his missing father', 'Telemachus resembles Ulysses wonderfully about the head and eyes', "Telemachus is the son of Ulysses and laments his father's mysterious disappearance", 'Ulysses is the father of Telemachus, thou

In [14]:
candidates_to_summarize, _, _ = neo4j_driver.execute_query(
    """MATCH (e:__Entity__) WHERE size(e.description) > 1 
    RETURN e.name AS entity_name, e.description AS description_list"""
)
summaries = []
for candidate in tqdm(candidates_to_summarize, desc="Summarizing entities"):
    messages = [
        {
            "role": "user",
            "content": ch07_tools.get_summarize_prompt(
                candidate["entity_name"], candidate["description_list"]
            ),
        },
    ]
    summary = chat(messages, model="gpt-4o")
    summaries.append({"entity": candidate["entity_name"], "summary": summary})

ch07_tools.import_entity_summary(summaries, neo4j_driver)

Summarizing entities:   0%|          | 0/30 [00:00<?, ?it/s]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
Telemachus is the son of the missing hero Ulysses, whom he strongly resembles around the head and eyes. He is a young man who has grown beyond infancy and now faces the burden of managing his household in his father's absence. Telemachus sits moodily among the suitors who have invaded his home, thinking about his brave but ill-starred father and hoping for his return, while simultaneously suffering from these chiefs who are eating up his house under the pretext of courting his mother Penelope.

When the goddess Minerva visits him disguised as the stranger Mentes, Telemachus serves as a gracious host, welcoming her, taking her spear, and conducting her to a decorated seat away from the suitors. He speaks quietly with the disguised goddess about the dire situation in his house, expressing concern about the cost of the suitors' entertainment and questioning her about her identity and origins. Though he offers hospita

Summarizing entities:   3%|▎         | 1/30 [00:11<05:28, 11.31s/it]

 action.⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
The Achaeans are Greek warriors who participated in the Trojan War and subsequently faced a difficult return journey from Troy. During their return, they suffered various ills inflicted upon them by the goddess Minerva as punishment or divine intervention. Among these returning warriors, Menelaus was notably the last to reach home safely.

The Achaeans maintain a connection to Ulysses and his family, as they would have honored the missing hero by building a mound over his ashes if he had died honorably in battle at Troy. In the present time of the narrative, Telemachus plans to call the Achaeans into assembly, indicating that they continue to serve as a governing or advisory body that can be convened to address important matters affecting their community and the household of Ulysses.

Summarizing entities:   7%|▋         | 2/30 [00:17<03:56,  8.44s/it]

...

 driving force in his heroic development.⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
Saturn is the father of the goddess Minerva and serves as a divine patriarch who is referred to as the King of kings. As Minerva's father, Saturn holds a position of supreme authority in the divine hierarchy, with his title indicating his status as the ruler over other divine rulers and his paramount importance

Summarizing entities:  97%|█████████▋| 29/30 [04:01<00:05,  5.14s/it]

 among the gods.⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
Euryclea is a good old woman and daughter of Ops who serves as a devoted servant in Telemachus' household. She has a special relationship with Telemachus, having nursed him as a baby and continuing to love him better than any other woman in the house. Euryclea performs various household duties for Telemachus, including lighting his way to his room and going before him with blazing torches, demonstrating her ongoing care and dedication to the young man she helpe

Summarizing entities: 100%|██████████| 30/30 [04:04<00:00,  8.16s/it]

d raise from infancy.

In [15]:
result, _, _ = neo4j_driver.execute_query(
    """MATCH (n:PERSON)
WHERE n.summary IS NULL
RETURN n.name AS name""")

In [16]:
result

[]

In [17]:
summary, _, _ = neo4j_driver.execute_query(
    """MATCH (n:PERSON)
WHERE n.name = "ORESTES"
RETURN n.summary AS summary""")
print(summary[0]['summary'])

Orestes is the son of Agamemnon who serves as a heroic example of filial duty and justified revenge. He grew up with the purpose of avenging his father's murder, ultimately killing Aegisthus, the man who had seduced Agamemnon's wife and murdered the king. Orestes is praised as a hero for this act of vengeance against his father's murderer, and his story is held up as an inspirational example for Telemachus, demonstrating how a son should honor his father's memory and seek justice against those who have wronged his family. His successful revenge against Aegisthus represents the fulfillment of both familial obligation and divine justice.



In [18]:
# 방법 2: 더 안전한 버전

def get_person_summary(person_name: str):
    """Get summary for a specific person from Neo4j"""
    try:
        query = """
        MATCH (n:PERSON)
        WHERE n.name = $person_name
        RETURN n.summary AS summary
        """
        
        records, _, _ = ch07_tools.neo4j_driver.execute_query(
            query, 
            person_name=person_name
        )
        
        if records and len(records) > 0:
            summary = records[0]['summary']
            if summary:
                return summary
            else:
                return f"No summary available for {person_name}"
        else:
            return f"Person '{person_name}' not found in database"
            
    except Exception as e:
        return f"Error querying database: {e}"

# 사용 예시
result = get_person_summary("ORESTES")
print(result)


Orestes is the son of Agamemnon who serves as a heroic example of filial duty and justified revenge. He grew up with the purpose of avenging his father's murder, ultimately killing Aegisthus, the man who had seduced Agamemnon's wife and murdered the king. Orestes is praised as a hero for this act of vengeance against his father's murderer, and his story is held up as an inspirational example for Telemachus, demonstrating how a son should honor his father's memory and seek justice against those who have wronged his family. His successful revenge against Aegisthus represents the fulfillment of both familial obligation and divine justice.



In [19]:
rels_to_summarize, _, _ = neo4j_driver.execute_query(
    """MATCH (s:__Entity__)-[r:RELATIONSHIP]-(t:__Entity__)
    WHERE id(s) < id(t)
    WITH s.name AS source, t.name AS target, 
           collect(r.description) AS description_list,
           count(*) AS count
    WHERE count > 1
    RETURN source, target, description_list"""
)
rel_summaries = []
for candidate in tqdm(rels_to_summarize, desc="Summarizing relationships"):
    entity_name = f"{candidate['source']} relationship to {candidate['target']}"
    messages = [
        {
            "role": "user",
            "content": ch07_tools.get_summarize_prompt(
                entity_name, candidate["description_list"]
            ),
        },
    ]
    summary = chat(messages, model="gpt-4o")
    rel_summaries.append({"source": candidate["source"], "target": candidate["target"], "summary": summary})

ch07_tools.import_rels_summary(summaries, neo4j_driver)

Summarizing relationships:   0%|          | 0/22 [00:00<?, ?it/s]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
The relationship between Telemachus and Minerva is one of divine mentorship and guidance, where the goddess serves as a protective and encouraging force in the young man's life. Minerva takes an active interest in Telemachus' development, putting heart into him and emboldening him during his father's prolonged absence. She disguises herself as Mentes to visit him directly, and though Telemachus initially believes he is speaking with a mortal guest, he comes to know in his heart that it had been the goddess herself.

During their encounter, Telemachus demonstrates proper hospitality by seeing Minerva first and going to greet her at the gate, taking her hand and spear in welcome. He conducts her to a seat as an honored guest and offers her hospitality, even wanting to give her a present as though she were a dear friend. Their conversation is intimate and confidential, with Telemachus speaking quietly to Minerva with

Summarizing relationships:   5%|▍         | 1/22 [00:08<02:49,  8.05s/it]

illing his destiny.⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
The relationship between Telemachus and Euryclea is one of deep maternal affection and devoted service, rooted in their shared history from his earliest years. Euryclea had nursed Telemachus as a baby, establishing a bond that has made her love him better than any other woman in the household. This special affection continues into his adulthood, where she serves him faithfully by performing various duties such as going before him with blazing torches to light his way. Their relationship represents the enduring connection between a devoted nurse and the child she helped raise, characterized by Euryclea's unwavering loyalty and maternal care

Summarizing relationships:   9%|▉         | 2/22 [00:12<01:58,  5.94s/it]

...

 legacy and authority.⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
The relationship between Aegisthus and Orestes is one of murderer and avenger, defined by a cycle of violence and retribution that spans generations. Aegisthus was the murderer of Orestes' father, creating a blood debt that demanded vengeance according to ancient codes of justice. This act established Aegisthus as Orestes' mortal enemy and set the stage for inevitable retaliation.

As Orestes grew to manhood, he was destined to take revenge on Aegisthus for his father's murder, representing both a filial duty and a demand for justice. When Orestes reached maturity, he fulfilled this obligation by killing Aegisthus in revenge, completing the cycle of retribution. This act of vengeance was not only expected but praised, as Orestes was commended for killing his father's murderer and restoring honor to his family.

The relationship exemplifies the classical theme of blood vengeance, where Aegisth

Summarizing relationships:  95%|█████████▌| 21/22 [02:11<00:07,  7.31s/it]

 act.⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
The relationship between Minerva and Saturn is that of daughter and father, as Minerva is the daughter of Saturn. This establishes their direct familial bond within the divine hierarchy, with Saturn serving as Minerva's paternal figure and Minerva holding her position as his offspring

Summarizing relationships: 100%|██████████| 22/22 [02:14<00:00,  6.10s/it]

 among the gods.

In [23]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (n:__Entity__)-[r:SUMMARIZED_RELATIONSHIP]-(m:__Entity__)
WHERE n.name = 'TELEMACHUS' AND m.name = 'MINERVA'
RETURN r.summary AS description
"""
)
print(data[0]["description"])

Minerva will put heart into Telemachus and embolden him


In [24]:
community_distribution = ch07_tools.calculate_communities(neo4j_driver)
print(f"There are {community_distribution['communityCount']} communities with distribution: {community_distribution['communityDistribution']}")

🔍 Calculating communities without GDS...
✅ Found 16 communities
   - Largest community: 97 nodes
   - Total nodes: 114
   - Total relationships: 133
There are 16 communities with distribution: {97: 1, 3: 1, 1: 14}


In [25]:
community_info, _, _ = neo4j_driver.execute_query(ch07_tools.community_info_query)

communities = []
for community in tqdm(community_info, desc="Summarizing communities"):
    messages = [
        {
            "role": "user",
            "content": ch07_tools.get_summarize_community_prompt(
                community["nodes"], community["rels"]
            ),
        },
    ]
    summary = chat(messages, model="gpt-4o")
    communities.append(
        {
            "community": json.loads(ch07_tools.extract_json(summary)),
            "communityId": community["communityId"],
            "nodes": [el["id"] for el in community["nodes"]],
        }
    )

neo4j_driver.execute_query(ch07_tools.import_community_query, data=communities)

Summarizing communities:   0%|          | 0/2 [00:00<?, ?it/s]

⚠️  Model 'gpt-4o' requested but using Bedrock instead (OpenAI disabled)
```json
{
    "title": "오디세이아의 핵심 인물들과 텔레마코스의 영웅적 여정",
    "summary": "이 커뮤니티는 호메로스의 오디세이아에 등장하는 주요 인물들과 그들 간의 복잡한 관계망을 보여줍니다. 실종된 영웅 율리시스의 아들 텔레마코스가 아버지를 찾기 위한 여정을 준비하는 과정을 중심으로, 가족 갈등, 신화적 존재들, 그리고 고대 그리스의 사회적 구조가 얽혀있습니다. 특히 폴리페모스와 율리시스의 과거 갈등이 현재 상황에 미치는 영향과 텔레마코스의 성장 서사가 주요 축을 이룹니다.",
    "rating": 8.7,
    "rating_explanation": "고전 서사시의 복합적인 서사 구조와 신화적 요소들이 체계적으로 연결되어 있으며, 영웅의 성장이라는 보편적 주제를 깊이 있게 다루고 있습니다.",
    "findings": [
        {
            "summary": "텔레마코스의 성장과 권위 확립",
            "explanation": "텔레마코스는 아버지의 부재로 인한 혼란 속에서 수동적인 청년에서 능동적인 지도자로 변화합니다. 미네르바의 조언을 받아 구혼자들에게 당당히 맞서고, 어머니에게 권위를 행사하며, 아버지를 찾기 위한 여정을 계획하는 등 이타카의 정당한 통치자로서의 자질을 보여줍니다."
        },
        {
            "summary": "과거 행동의 현재적 결과와 신화적 운명",
            "explanation": "율리시스가 폴리페모스의 눈을 멀게 한 행위가 넵튠의 분노를 사서 현재의 고난을 초래했듯이, 과거의 행동이 현재와 미래에 미치는 영향이 강조됩니다. 이는 고대 그리스 신화에서 중요한 주제인 운명과 신들의 개입, 그리고 행동에 따른 필연적 결과를 보

Summarizing communities:   0%|          | 0/2 [00:10<?, ?it/s]

여줍니다."
        }
    ]
}
```

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [ ]:
data, _, _ = neo4j_driver.execute_query(
    """MATCH (c:__Community__)
WITH c, count {(c)<-[:IN_COMMUNITY]-()} AS size
ORDER BY size DESC LIMIT 1
RETURN c.title AS title, c.summary AS summary
"""
)
print(data[0]["title"])
print(data[0]["summary"])

In [ ]:
def global_retriever(query: str, rating_threshold: float = 5) -> str:
    community_data, _, _ = neo4j_driver.execute_query(
        """
    MATCH (c:__Community__)
    WHERE c.rating >= $rating
    RETURN c.summary AS summary
    """,
        rating=rating_threshold,
    )
    print(f"Got {len(community_data)} community summaries")
    intermediate_results = []
    for community in tqdm(community_data, desc="Processing communities"):
        intermediate_messages = [
            {
                "role": "system",
                "content": ch07_tools.get_map_system_prompt(community["summary"]),
            },
            {
                "role": "user",
                "content": query,
            },
        ]
        intermediate_response = chat(intermediate_messages, model="gpt-4o")
        intermediate_results.append(intermediate_response)

    final_messages = [
        {
            "role": "system",
            "content": ch07_tools.get_reduce_system_prompt(intermediate_results),
        },
        {"role": "user", "content": query},
    ]
    summary = chat(final_messages, model="gpt-4o")
    return summary

In [ ]:
print(global_retriever("What is this story about?"))

In [ ]:
entities, _, _ = neo4j_driver.execute_query(
    """
MATCH (e:__Entity__)
RETURN e.summary AS summary, e.name AS name
"""
)
data = [{"name": el["name"], "embedding": embed(el["summary"])[0]} for el in entities]
neo4j_driver.execute_query(
    """
UNWIND $data AS row
MATCH (e:__Entity__ {name: row.name})
CALL db.create.setNodeVectorProperty(e, 'embedding', row.embedding)
""",
    data=data,
)

neo4j_driver.execute_query(
    """
CREATE VECTOR INDEX entities IF NOT EXISTS
FOR (n:__Entity__)
ON (n.embedding)
""",
    data=data,
)


In [ ]:
local_search_query = """
CALL db.index.vector.queryNodes('entities', $k, $embedding)
YIELD node, score
WITH collect(node) as nodes
WITH collect {
    UNWIND nodes as n
    MATCH (n)<-[:HAS_ENTITY]->(c:__Chunk__)
    WITH c, count(distinct n) as freq
    RETURN c.text AS chunkText
    ORDER BY freq DESC
    LIMIT $topChunks
} AS text_mapping,
collect {
    UNWIND nodes as n
    MATCH (n)-[:IN_COMMUNITY]->(c:__Community__)
    WITH c, c.rank as rank, c.weight AS weight
    RETURN c.summary 
    ORDER BY rank, weight DESC
    LIMIT $topCommunities
} AS report_mapping,
collect {
    UNWIND nodes as n
    MATCH (n)-[r:SUMMARIZED_RELATIONSHIP]-(m) 
    WHERE m IN nodes
    RETURN r.summary AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topInsideRels
} as insideRels,
collect {
    UNWIND nodes as n
    RETURN n.summary AS descriptionText
} as entities
RETURN {Chunks: text_mapping, Reports: report_mapping, 
       Relationships: insideRels, 
       Entities: entities} AS text
"""

In [ ]:
k_entities = 5

topChunks = 3
topCommunities = 3
topInsideRels = 3


def local_search(query: str) -> str:
    context, _, _ = neo4j_driver.execute_query(
        local_search_query,
        embedding=embed(query)[0],
        topChunks=topChunks,
        topCommunities=topCommunities,
        topInsideRels=topInsideRels,
        k=k_entities,
    )
    context_str = str(context[0]["text"])
    local_messages = [
        {
            "role": "system",
            "content": ch07_tools.get_local_system_prompt(context_str),
        },
        {
            "role": "user",
            "content": query,
        },
    ]
    final_answer = chat(local_messages, model="gpt-4o")
    return final_answer


In [ ]:
print(local_search("Who is Ulysses?"))